## Kanellaki Maria Anna 

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords, dependency_treebank
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from nltk import word_tokenize, pos_tag_sents
from collections import Counter
from nltk.probability import ConditionalFreqDist
from sklearn.ensemble import RandomForestClassifier

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marianna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marianna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/marianna/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Create data sets 

In [2]:
#creates train set
train_set = pd.read_csv('data/train.csv')
train = pd.DataFrame(train_set)
train

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."


In [3]:
#creates test set
test_set = pd.read_csv('data/impermium_verification_set.csv')
test = pd.DataFrame(test_set)
test

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,NaN,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,NaN,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,NaN,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,NaN,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,NaN,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,NaN,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,NaN,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [4]:
#creates labeled set
labeled_set = pd.read_csv('data/impermium_verification_labels.csv')
labeled = pd.DataFrame(labeled_set)
labeled

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,1,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


## Preprocess data function

In [5]:
def preprocessData(df):
    # remove uppercase letters
    df['Comment'] = df['Comment'].apply(lambda s:s.lower() if type(s) == str else s)
    
    #remove URLs
    df['Comment'] = df['Comment'].replace(to_replace =r'^https?:\/\/.*[\r\n]*', value = '', regex = True)
    
    # remove numbers
    df['Comment'] = df['Comment'].replace(to_replace =r'\d+', value = '', regex = True)
    
    #remove \n, \u0111
    df['Comment'] = df['Comment'].replace(to_replace ='\n', value = '', regex = True)
    df['Comment'] = df['Comment'].replace(to_replace ='\u0111', value = '', regex = True)
    
    # remove symbols
    df['Comment'] = df['Comment'].replace(to_replace =r'([^\s\w]|_)+', value = '', regex = True)

    return df

## Preprocess data for all dataFrames 

In [6]:
train = preprocessData(train)
train

,Insult,Date,Comment
0,1,20120618192155Z,you fuck your dad
1,0,20120528192215Z,i really dont understand your pointxa it seems...
2,0,NaN,axcxamajority of canadians can and has been wr...
3,0,NaN,listen if you dont wanna get married to a man ...
4,0,20120619094753Z,cxec buean xuuedng uubueddng biuecu txecnh cx...
...,...,...,...
3942,1,20120502172717Z,you are both morons and that is never happening
3943,0,20120528164814Z,many toolbars include spell check like yahoo f...
3944,0,20120620142813Z,lambeauorwrigleyxaxakmossxansioux falls sd i t...
3945,0,20120528205648Z,how about felix he is sure turning into one he...


In [7]:
test = preprocessData(test)
test

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,NaN,20120531215447Z,youre idiot,PrivateTest
2,3,NaN,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,NaN,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,NaN,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,NaN,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,NaN,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,NaN,20120531045826Z,ballreal where are you with your miami gayness,PrivateTest


In [8]:
labeled = preprocessData(labeled)
labeled

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,1,20120531215447Z,youre idiot,PrivateTest
2,3,1,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,1,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,1,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,1,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,0,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,1,20120531045826Z,ballreal where are you with your miami gayness,PrivateTest


## Vectorize data frames

In [9]:
#with unigrams without any optimization for now

vectorizer = CountVectorizer()
vectorizer.fit(train['Comment'])

train_vec = vectorizer.transform(train['Comment'])
print(vectorizer.get_feature_names())

['aaaah', 'aaahhh', 'aac', 'aamir', 'aap', 'aarongmyers', 'ab', 'abacha', 'abandon', 'abandoned', 'abandonment', 'abc', 'abe', 'abeds', 'abel', 'aberdeen', 'abetted', 'abide', 'abiding', 'abigail', 'abilities', 'ability', 'abilitya', 'abilityn', 'abit', 'able', 'abnormal', 'abolish', 'abolition', 'abomination', 'abominations', 'abort', 'aborted', 'abortifacients', 'abortion', 'abortionneven', 'abortions', 'abortionsnnalso', 'about', 'aboutn', 'aboutnot', 'aboutxauuuauua', 'aboutxaxa', 'above', 'abraham', 'abroad', 'abroadxcxanfacts', 'abruptly', 'abs', 'abscam', 'absence', 'abslutely', 'absolute', 'absolutejoke', 'absolutely', 'abstaining', 'absurd', 'absurdly', 'absurdumnatt', 'absurt', 'aburrido', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abxcxa', 'abxxv', 'abysmal', 'ac', 'academic', 'acc', 'accentblacks', 'accept', 'acceptable', 'accepted', 'accepting', 'access', 'accessible', 'accessories', 'accident', 'accidental', 'accidentally', 'accidents', 'accolades', 'accomplish',

In [10]:
test_vec = vectorizer.transform(test['Comment'])
print(vectorizer.get_feature_names())

['aaaah', 'aaahhh', 'aac', 'aamir', 'aap', 'aarongmyers', 'ab', 'abacha', 'abandon', 'abandoned', 'abandonment', 'abc', 'abe', 'abeds', 'abel', 'aberdeen', 'abetted', 'abide', 'abiding', 'abigail', 'abilities', 'ability', 'abilitya', 'abilityn', 'abit', 'able', 'abnormal', 'abolish', 'abolition', 'abomination', 'abominations', 'abort', 'aborted', 'abortifacients', 'abortion', 'abortionneven', 'abortions', 'abortionsnnalso', 'about', 'aboutn', 'aboutnot', 'aboutxauuuauua', 'aboutxaxa', 'above', 'abraham', 'abroad', 'abroadxcxanfacts', 'abruptly', 'abs', 'abscam', 'absence', 'abslutely', 'absolute', 'absolutejoke', 'absolutely', 'abstaining', 'absurd', 'absurdly', 'absurdumnatt', 'absurt', 'aburrido', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abxcxa', 'abxxv', 'abysmal', 'ac', 'academic', 'acc', 'accentblacks', 'accept', 'acceptable', 'accepted', 'accepting', 'access', 'accessible', 'accessories', 'accident', 'accidental', 'accidentally', 'accidents', 'accolades', 'accomplish',

# Bayes without any optimizations

In [11]:
mnb = MultinomialNB(alpha=0)                    #without laplace smoothing
mnb.fit(train_vec.toarray(), train['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

#medium scores

/home/marianna/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

           0       0.58      0.64      0.61      1158
           1       0.57      0.51      0.54      1077

    accuracy                           0.58      2235
   macro avg       0.58      0.58      0.58      2235
weighted avg       0.58      0.58      0.58      2235



## Lemmatize data frames

In [12]:
#lemmatization in all data frames 'comment' column
lemmatizer = WordNetLemmatizer()

train_l = train.copy()  #copies data frames so that the original ones do not change
test_l = test.copy()

train_l['Comment'] = train['Comment'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
test_l['Comment'] = test['Comment'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

#create new vectors with lemmatization
vectorizer = CountVectorizer()
vectorizer.fit(train_l['Comment'])

train_vec = vectorizer.transform(train_l['Comment'])
test_vec = vectorizer.transform(test_l['Comment'])

## Bayes with lemmatization

In [13]:
mnb.fit(train_vec.toarray(), train_l['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

#scores don't change much

/home/marianna/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

           0       0.59      0.64      0.61      1158
           1       0.57      0.52      0.54      1077

    accuracy                           0.58      2235
   macro avg       0.58      0.58      0.58      2235
weighted avg       0.58      0.58      0.58      2235



## Remove stop words 

In [14]:
# removes them from all data frames 'comment' column
stop = stopwords.words('english')

train_s = train.copy()          #copies data frames so that the original ones do not change
test_s = test.copy()

train_s['Comment'] = train['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_s['Comment'] = test['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#create new vectors without stopwords
vectorizer = CountVectorizer()
vectorizer.fit(train_s['Comment'])

train_vec = vectorizer.transform(train_s['Comment'])
test_vec = vectorizer.transform(test_s['Comment'])

## Bayes without stopwords

In [15]:
mnb.fit(train_vec.toarray(), train_s['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

#scores increase slightly, a bit more for insult=1

/home/marianna/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

           0       0.61      0.78      0.68      1158
           1       0.66      0.46      0.54      1077

    accuracy                           0.63      2235
   macro avg       0.64      0.62      0.61      2235
weighted avg       0.63      0.63      0.62      2235



## Create vectors of bigrams

In [16]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
vectorizer.fit(train['Comment'])

train_vec = vectorizer.transform(train['Comment'])
test_vec = vectorizer.transform(test['Comment'])

## Naive Bayes with bigrams

In [17]:
mnb.fit(train_vec.toarray(), train['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

# scores increase slightly

/home/marianna/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

           0       0.59      0.74      0.65      1158
           1       0.61      0.45      0.52      1077

    accuracy                           0.60      2235
   macro avg       0.60      0.59      0.59      2235
weighted avg       0.60      0.60      0.59      2235



## Create new simple unigram vectors 

In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(train['Comment'])

train_vec = vectorizer.transform(train['Comment'])
test_vec = vectorizer.transform(test['Comment'])

## Naive Bayes with Laplace smoothing

In [19]:
mnb = MultinomialNB()                         #default alpha=1 with laplace smoothing
mnb.fit(train_vec.toarray(), train['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

#scores increase significantly especially for insult=1

              precision    recall  f1-score   support

           0       0.65      0.85      0.73      1158
           1       0.75      0.50      0.60      1077

    accuracy                           0.68      2235
   macro avg       0.70      0.67      0.67      2235
weighted avg       0.70      0.68      0.67      2235



## Part of speech features

In [21]:
tok = nltk.tokenize.WhitespaceTokenizer()

# creates new column in train with comment's words' pos tags
train['POS_Comment'] = None
train['POS_Comment'] = pos_tag_sents(train['Comment'].apply(word_tokenize).tolist())

#creates new columns for each part of speech required
train['fractionAdverbs'] = None
train['fractionVerbs'] = None
train['fractionAdjectives'] = None
train['fractionNouns'] = None

#parts of speech that are needed
nouns = ['NN','NNS','NNP','NNPS']
adverbs = ['RB','RBR','RBS','RP']
verbs = ['VB','VBD','VBG','VBN','VBP','VBZ','MD']
adjectives = ['JJ','JJR','JJS']
    
#calculate words for each comment    
for index, row in train.iterrows():
    count_all = Counter(tag for word,tag in row['POS_Comment'])
    count_n = Counter(tag for word,tag in row['POS_Comment'] if tag in nouns)
    count_v = Counter(tag for word,tag in row['POS_Comment'] if tag in verbs)
    count_adj = Counter(tag for word,tag in row['POS_Comment'] if tag in adjectives)
    count_adv = Counter(tag for word,tag in row['POS_Comment'] if tag in adverbs)
    
    total = sum(count_all.values())
    
    # add each frequency to the new columns
    train.loc[train.index == index,'fractionNouns'] = sum(count_n.values()) / total
    train.loc[train.index == index,'fractionVerbs'] = sum(count_v.values()) / total
    train.loc[train.index == index,'fractionAdjectives'] = sum(count_adj.values()) / total
    train.loc[train.index == index,'fractionAdverbs'] = sum(count_adv.values()) / total
    
train

,Insult,Date,Comment,POS_Comment,fractionAdverbs,fractionVerbs,fractionAdjectives,fractionNouns
0,1,20120618192155Z,you fuck your dad,"[(you, PRP), (fuck, VBP), (your, PRP$), (dad, ...",0,0.25,0,0.25
1,0,20120528192215Z,i really dont understand your pointxa it seems...,"[(i, NN), (really, RB), (dont, JJ), (understan...",0.0666667,0.266667,0.0666667,0.266667
2,0,NaN,axcxamajority of canadians can and has been wr...,"[(axcxamajority, NN), (of, IN), (canadians, NN...",0.078125,0.21875,0.078125,0.234375
3,0,NaN,listen if you dont wanna get married to a man ...,"[(listen, VB), (if, IN), (you, PRP), (dont, VB...",0.0338983,0.305085,0.0847458,0.118644
4,0,20120619094753Z,cxec buean xuuedng uubueddng biuecu txecnh cx...,"[(cxec, JJ), (buean, JJ), (xuuedng, NNP), (uub...",0,0.0131579,0.263158,0.723684
...,...,...,...,...,...,...,...,...
3942,1,20120502172717Z,you are both morons and that is never happening,"[(you, PRP), (are, VBP), (both, DT), (morons, ...",0.111111,0.333333,0,0.111111
3943,0,20120528164814Z,many toolbars include spell check like yahoo f...,"[(many, JJ), (toolbars, NNS), (include, VBP), ...",0.0769231,0.346154,0.0769231,0.153846
3944,0,20120620142813Z,lambeauorwrigleyxaxakmossxansioux falls sd i t...,"[(lambeauorwrigleyxaxakmossxansioux, NN), (fal...",0.0416667,0.291667,0.0416667,0.333333
3945,0,20120528205648Z,how about felix he is sure turning into one he...,"[(how, WRB), (about, RB), (felix, NN), (he, PR...",0.0833333,0.222222,0.0277778,0.277778


## Same procedure for test data frame 

In [23]:
tok = nltk.tokenize.WhitespaceTokenizer()

# creates new column in train with comment's words' pos tags
test['POS_Comment'] = None
test['POS_Comment'] = pos_tag_sents(test['Comment'].apply(word_tokenize).tolist())

#creates new columns for each part of speech required
test['fractionAdverbs'] = None
test['fractionVerbs'] = None
test['fractionAdjectives'] = None
test['fractionNouns'] = None
    
#calculate words for each comment    
for index, row in train.iterrows():
    count_all = Counter(tag for word,tag in row['POS_Comment'])
    count_n = Counter(tag for word,tag in row['POS_Comment'] if tag in nouns)
    count_v = Counter(tag for word,tag in row['POS_Comment'] if tag in verbs)
    count_adj = Counter(tag for word,tag in row['POS_Comment'] if tag in adjectives)
    count_adv = Counter(tag for word,tag in row['POS_Comment'] if tag in adverbs)
    
    total = sum(count_all.values())
    
    # add each frequency to the new columns
    test.loc[test.index == index,'fractionNouns'] = sum(count_n.values()) / total
    test.loc[test.index == index,'fractionVerbs'] = sum(count_v.values()) / total
    test.loc[test.index == index,'fractionAdjectives'] = sum(count_adj.values()) / total
    test.loc[test.index == index,'fractionAdverbs'] = sum(count_adv.values()) / total
    
test

,id,Insult,Date,Comment,Usage,POS_Comment,fractionAdverbs,fractionVerbs,fractionAdjectives,fractionNouns
0,1,NaN,20120603163526Z,like this if you are a tribe fan,PrivateTest,"[(like, IN), (this, DT), (if, IN), (you, PRP),...",0,0.25,0,0.25
1,2,NaN,20120531215447Z,youre idiot,PrivateTest,"[(youre, NN), (idiot, NN)]",0.0666667,0.266667,0.0666667,0.266667
2,3,NaN,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest,"[(i, NN), (am, VBP), (a, DT), (woman, NN), (ba...",0.078125,0.21875,0.078125,0.234375
3,4,NaN,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest,"[(wow, NN), (you, PRP), (benefitted, VBD), (so...",0.0338983,0.305085,0.0847458,0.118644
4,5,NaN,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest,"[(haha, NN), (green, JJ), (me, PRP), (red, JJ)...",0,0.0131579,0.263158,0.723684
...,...,...,...,...,...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest,"[(fuckin, JJ), (lame, NN), (come, VBN), (on, I...",0,0.666667,0,0
2231,2232,NaN,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest,"[(you, PRP), (shut, VBP), (your, PRP$), (ignor...",0.0666667,0.133333,0,0.333333
2232,2233,NaN,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest,"[(sweetie, NN), (pie, NN), (is, VBZ), (looking...",0.08,0.22,0.08,0.26
2233,2234,NaN,20120531045826Z,ballreal where are you with your miami gayness,PrivateTest,"[(ballreal, NN), (where, WRB), (are, VBP), (yo...",0.05,0.25,0.2,0.25


## vectors with TF-IDF features

In [24]:
# in bigrams
train_tf = TfidfVectorizer(analyzer="char", ngram_range=(2,2)).fit_transform(train['Comment'])
print(train_tf)

  (0, 30)	0.29559314900332245
  (0, 108)	0.3513115004964081
  (0, 3)	0.20907786244569307
  (0, 467)	0.18068757195060695
  (0, 564)	0.2239614738717422
  (0, 24)	0.18971725201170858
  (0, 292)	0.2525997471390764
  (0, 91)	0.2572543582794901
  (0, 550)	0.2961160975494616
  (0, 181)	0.31819790039113305
  (0, 5)	0.19994961221207097
  (0, 547)	0.2043355333175318
  (0, 418)	0.30929581295376374
  (0, 666)	0.36107360677416794
  (1, 191)	0.1100510877670391
  (1, 468)	0.10404439257233937
  (1, 415)	0.08135303694863025
  (1, 14)	0.07572628906718176
  (1, 153)	0.17004863478158955
  (1, 323)	0.08626886944673542
  (1, 435)	0.12123032862993272
  (1, 439)	0.15496706481234096
  (1, 42)	0.14195870264927693
  (1, 186)	0.08052071732537464
  (1, 377)	0.15430035924213875
  :	:
  (3946, 152)	0.12551323333844241
  (3946, 112)	0.04568486278959821
  (3946, 374)	0.18545753473160898
  (3946, 20)	0.049717951511273645
  (3946, 521)	0.1638964098057826
  (3946, 411)	0.033834246243140936
  (3946, 122)	0.186761390676155

In [25]:
test_tf = TfidfVectorizer(analyzer="char", ngram_range=(2,2)).fit_transform(test['Comment'])
print(test_tf)

  (0, 40)	0.13828490578570066
  (0, 155)	0.2459724757319206
  (0, 5)	0.1544059851267783
  (0, 58)	0.1841816591325612
  (0, 225)	0.3582995751731151
  (0, 435)	0.21147084419266493
  (0, 497)	0.23083038622621854
  (0, 26)	0.15184749345991352
  (0, 431)	0.11310506244650737
  (0, 44)	0.13392334182803245
  (0, 0)	0.2147785198324995
  (0, 506)	0.11166430956099191
  (0, 389)	0.08925847943971228
  (0, 609)	0.09358368499732835
  (0, 24)	0.1076908491279337
  (0, 154)	0.1909265557948937
  (0, 229)	0.2560031342645791
  (0, 8)	0.14647171934698544
  (0, 453)	0.1258470467186774
  (0, 242)	0.17594287665178948
  (0, 206)	0.17916291128024575
  (0, 488)	0.1349037999840233
  (0, 19)	0.2519463172226235
  (0, 127)	0.28291858606388287
  (0, 269)	0.19767047557660536
  :	:
  (2234, 237)	0.15839010936554732
  (2234, 18)	0.08132538300200337
  (2234, 426)	0.08191479587518988
  (2234, 553)	0.13622656796401847
  (2234, 382)	0.09539579245536968
  (2234, 101)	0.08634009358631553
  (2234, 345)	0.10344240192767676
  (22

## SVM

In [27]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'linear']}

grid = GridSearchCV(svm.SVC(), param_grid, scoring = ['accuracy', 'precision_micro', 'recall_micro', 'f1_micro' ],
                       cv = 10,
                       refit= 'accuracy',
                       verbose = 1,
                       n_jobs = -1)

clf = svm.SVC(kernel = 'linear')
clf.fit(train_vec.toarray(), train['Insult'])

pred = clf.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))


              precision    recall  f1-score   support

           0       0.67      0.85      0.75      1158
           1       0.78      0.56      0.65      1077

    accuracy                           0.71      2235
   macro avg       0.73      0.71      0.70      2235
weighted avg       0.72      0.71      0.70      2235



## Random forests

In [29]:
clf=RandomForestClassifier(n_estimators=100)

clf.fit(train_vec.toarray(), train['Insult'])

pred=clf.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))


              precision    recall  f1-score   support

           0       0.59      0.95      0.72      1158
           1       0.83      0.29      0.43      1077

    accuracy                           0.63      2235
   macro avg       0.71      0.62      0.58      2235
weighted avg       0.70      0.63      0.58      2235



## Experimentation 

In [30]:
# clean data

#lemmatization
train['Comment'] = train['Comment'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
test['Comment'] = test['Comment'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

#remove stopwords
train_s['Comment'] = train['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_s['Comment'] = test['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#didnt use bigrams. they 

## Naive Bayes with my optimizations 

In [31]:
mnb = MultinomialNB()                         #default alpha=1 with laplace smoothing
mnb.fit(train_vec.toarray(), train['Insult'])

pred = mnb.predict(test_vec.toarray())
print(classification_report(labeled['Insult'],pred))

#scores increase significantly especially for insult=1

              precision    recall  f1-score   support

           0       0.65      0.85      0.73      1158
           1       0.75      0.50      0.60      1077

    accuracy                           0.68      2235
   macro avg       0.70      0.67      0.67      2235
weighted avg       0.70      0.68      0.67      2235

